In [1]:
from src.features.videos.objects import object as object_helper
from src.models.lda import lda as lda_helper
from src import constants

import pandas as pd
import numpy as np

import psycopg2

%load_ext autoreload
%autoreload 2

/Users/claasmeiners/.virtualenvs/video_article_retrieval/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
object_occurence_rate = object_helper.divide_by_duration(object_helper.get_probability_sum())
object_occurence_rate.head()

features_connection = psycopg2.connect(database="video_article_retrieval", user="postgres")
average_topics = pd.read_sql("SELECT * FROM average_topics", con=features_connection)

data = pd.merge(average_topics, object_occurence_rate,  how='right', left_on=['id','platform'], right_on = ['id','platform'])
data.head()

,id,platform,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,...,sink,refrigerator,book,clock,vase,scissors,teddy_bear,hair_drier,toothbrush,duration
0,wsbtv/10157004889480695,facebook,0.000000,0.653323,0.0,0.000,0.022118,0.000000,0.125464,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,236288
1,KHOU11/10157855122244062,facebook,0.002818,0.927415,0.0,0.000,0.000000,0.043440,0.005280,0.018187,...,0.0,0.000000,0.000000,0.000147,0.0,0.0,0.0,0.0,0.0,729216
2,100013660111371/486703935128273,facebook,0.000000,0.629809,0.0,0.051,0.211798,0.093693,0.012093,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,779904
3,1488021131510328/1994413240871112,facebook,0.000000,0.453624,0.0,0.000,0.000000,0.132679,0.366747,0.040566,...,0.0,0.000003,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,268051
4,ABC15/10155973576596359,facebook,0.000000,0.000000,0.0,0.000,0.000000,0.223820,0.428964,0.259963,...,0.0,0.000000,0.000041,0.000000,0.0,0.0,0.0,0.0,0.0,74153


In [3]:
topics = lda_helper.get_topic_columns(10)
objects = [label.replace(" ", "_") for label in constants.COCO_CLASS_NAMES]
data[topics].head()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,0.000000,0.653323,0.0,0.000,0.022118,0.000000,0.125464,0.000000,0.197165,0.000000
1,0.002818,0.927415,0.0,0.000,0.000000,0.043440,0.005280,0.018187,0.000000,0.000000
2,0.000000,0.629809,0.0,0.051,0.211798,0.093693,0.012093,0.000000,0.000000,0.000000
3,0.000000,0.453624,0.0,0.000,0.000000,0.132679,0.366747,0.040566,0.000000,0.000000
4,0.000000,0.000000,0.0,0.000,0.000000,0.223820,0.428964,0.259963,0.020975,0.062212


In [4]:
data[objects].head()

,person,bicycle,car,motorbike,aeroplane,bus,train,truck,boat,traffic_light,...,toaster,sink,refrigerator,book,clock,vase,scissors,teddy_bear,hair_drier,toothbrush
0,0.000984,0.000000,0.000018,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.002102,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000147,0.0,0.0,0.0,0.0,0.0
2,0.000972,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.001395,0.000002,0.000322,0.000007,0.0,0.0,0.000003,0.000039,0.0,0.0,...,0.0,0.0,0.000003,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.000547,0.000000,0.000257,0.000000,0.0,0.0,0.000009,0.000245,0.0,0.0,...,0.0,0.0,0.000000,0.000041,0.000000,0.0,0.0,0.0,0.0,0.0


In [17]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# models
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, RidgeCV, BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

x = data[objects]
y = data[topics]

x = preprocessing.scale(x)
y = preprocessing.scale(y)

print(len(x[0]))

models = [
    ("Dummy", DummyRegressor(strategy='mean')),
    #("Linear", LinearRegression()),
    #("RandomForest", RandomForestRegressor()),
    #("Kneighbors", KNeighborsRegressor()),
    #("RidgeCV", RidgeCV()),
    #("GaussianProcess", GaussianProcessRegressor()),
    ("Neural Network", MLPRegressor(hidden_layer_sizes=(60, )))
]

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mean_absolute_percentage_error(estimator, x, y):
    y_pred = estimator.predict(x)
    return np.mean(np.abs((y - y_pred) / y)) * 100

for name, model in models:
    print(name.center(20,"="))
    model.fit(x , y)
    scores = cross_val_score(model, x, y, cv=10, scoring='neg_mean_absolute_error')
    # print("MAPE: %.4f" % mape(y_test, y_pred))
    # print('Variance score: %.4f' % r2_score(y_test, y_pred))
    print("Score: %0.3f (+/- %0.3f)" % (-scores.mean(), scores.std() * 2))

80
=======Dummy========
Score: 0.631 (+/- 0.028)
===Neural Network===


/Users/claasmeiners/.virtualenvs/video_article_retrieval/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/claasmeiners/.virtualenvs/video_article_retrieval/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/claasmeiners/.virtualenvs/video_article_retrieval/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/claasmeiners/.virtualenvs/video_article_retrieval/lib/python3.5/site-packages/sklearn/neural_network/multila

Score: 0.696 (+/- 0.076)


/Users/claasmeiners/.virtualenvs/video_article_retrieval/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [136]:
x.values[14]

array([5.62716089e-04, 0.00000000e+00, 9.59236980e-06, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.20622277e-06, 0.00000000e+00, 0.00000000e+00, 1.56407074e-06,
       0.00000000e+00, 0.00000000e+00, 2.63450340e-06, 0.00000000e+00,
       8.75573100e-07, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.59765002e-06, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [137]:
y.values[14]

array([0.       , 0.0168495, 0.       , 0.       , 0.       , 0.128777 ,
       0.321615 , 0.       , 0.5296975, 0.       ])

In [138]:
data.iloc[14]

id               bretthofflandkstp/2084760468511263
platform                                   facebook
topic_0                                           0
topic_1                                   0.0168495
topic_2                                           0
topic_3                                           0
topic_4                                           0
topic_5                                    0.128777
topic_6                                    0.321615
topic_7                                           0
topic_8                                    0.529698
topic_9                                           0
person                                  0.000562716
bicycle                                           0
car                                     9.59237e-06
motorbike                                         0
aeroplane                                         0
bus                                               0
train                                             0
truck       

In [24]:
regr = LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

NameError: name 'linear_model' is not defined

In [ ]:
models = [linear_model.LinearRegression()]
for model in models:
    model.fit(data[objects], data[topics])
    scores = cross_val_score(model, data[objects], data[topics], cv=5)
    # print('Coefficients: \n', regr.coef_)
    # print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))
    # print(model.score(X_test, y_test))
    print(scores)